<a href="https://colab.research.google.com/github/Hippopotamus0308/torch2-test/blob/feat-basic-test/torch2_test_compile_mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install --pre torch --force-reinstall --index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.6/532.6 KB 26.7 MB/s eta 0:00:00
  Attempting uninstall: mpmath
    Found existing installation: mpmath 1.2.1
    Uninstalling mpmath-1.2.1:
      Successfully uninstalled mpmath-1.2.1
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.7.1
    Uninstalling sympy-1.7.1:
      Successfully uninstalled sympy-1.7.1
  Attempting uninstall: networkx
    Found existing installation

In [2]:
import torch
import numpy as np
import torch._dynamo
from typing import List
import time

def timed(fn):
    start = time.time()
    result = fn()
    end = time.time()
    time_cnt = end - start
    #print(f"{printer}, time: {time_cnt}")
    return result, time_cnt


def generate_data(b):
    return (
        torch.randn(b, 3, 128, 128).to(torch.float32),
        torch.randint(1000, (b,)),
    )

In [4]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
opt_model_default = torch.compile(model, mode="default")
opt_model_reduce_overhead = torch.compile(model, mode="reduce-overhead")
opt_model_max_autotune = torch.compile(model, mode="max-autotune")

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [6]:
with torch.no_grad():
  opt_model_default(generate_data(2)[0])

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 1368, in aot_wrapper_dedupe
    fw_metadata, _out = run_functionalized_fw_and_collect_metadata(flat_fn)(
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 569, in inner
    flat_f_outs = f(*flat_f_args)
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 2415, in functional_call
    out = Interpreter(mod).run(*args[params_len:], **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 136, in run
    self.env[node] = self.run_node(node)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 177, in run_node
    return getattr(self, n.op)(n.target, args, kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 294, in call_module
    return submod(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torc

In [7]:
def test(cnt):
  time_no_opt = []
  time_default = []
  time_reduce_overhead = []
  time_max_autotune = []

  ## warm up
  for i in range(5):
    with torch.no_grad():
      model(generate_data(cnt)[0])
      opt_model_default(generate_data(cnt)[0])
      opt_model_reduce_overhead(generate_data(cnt)[0])
      opt_model_max_autotune(generate_data(cnt)[0])

  for i in range(10):
    with torch.no_grad():
      _, time1 = timed(lambda:model(generate_data(cnt)[0]))
      _, time2 = timed(lambda:opt_model_default(generate_data(cnt)[0]))
      _, time3 = timed(lambda:opt_model_reduce_overhead(generate_data(cnt)[0]))
      _, time4 = timed(lambda:opt_model_max_autotune(generate_data(cnt)[0]))
    time_no_opt.append(time1)
    time_default.append(time2)
    time_reduce_overhead.append(time3)
    time_max_autotune.append(time4)   

  no_opt_median_time = np.median(time_no_opt)
  default_opt_median_time = np.median(time_default)
  ro_median_time = np.median(time_reduce_overhead)
  ma_median_time = np.median(time_max_autotune)

  no_opt_mean_time = np.mean(time_no_opt)
  default_opt_mean_time = np.mean(time_default)
  ro_mean_time = np.mean(time_reduce_overhead)
  ma_mean_time = np.mean(time_max_autotune)

  print("-------------Median Time---------------")
  print(f"no opt median time: {no_opt_median_time}")
  print(f"mode = defualt: {default_opt_median_time}")
  print(f"mode = reduce overhead: {ro_median_time}")
  print(f"mode = max autotune: {ma_median_time}")

  print("-------------Mean Time---------------")
  print(f"no opt mean time: {no_opt_mean_time}")
  print(f"mode = defualt: {default_opt_mean_time}")
  print(f"mode = reduce overhead: {ro_mean_time}")
  print(f"mode = max autotune: {ma_mean_time}")  

In [8]:
test(1)

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 1368, in aot_wrapper_dedupe
    fw_metadata, _out = run_functionalized_fw_and_collect_metadata(flat_fn)(
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 569, in inner
    flat_f_outs = f(*flat_f_args)
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 2415, in functional_call
    out = Interpreter(mod).run(*args[params_len:], **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 136, in run
    self.env[node] = self.run_node(node)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 177, in run_node
    return getattr(self, n.op)(n.target, args, kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 294, in call_module
    return submod(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torc

-------------Median Time---------------
no opt median time: 0.03492546081542969
mode = defualt: 0.027805447578430176
mode = reduce overhead: 0.028104186058044434
mode = max autotune: 0.028142809867858887
-------------Mean Time---------------
no opt mean time: 0.03588788509368897
mode = defualt: 0.02912609577178955
mode = reduce overhead: 0.02819325923919678
mode = max autotune: 0.02925412654876709


In [9]:
test(8)

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 1368, in aot_wrapper_dedupe
    fw_metadata, _out = run_functionalized_fw_and_collect_metadata(flat_fn)(
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 569, in inner
    flat_f_outs = f(*flat_f_args)
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 2415, in functional_call
    out = Interpreter(mod).run(*args[params_len:], **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 136, in run
    self.env[node] = self.run_node(node)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 177, in run_node
    return getattr(self, n.op)(n.target, args, kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 294, in call_module
    return submod(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torc

-------------Median Time---------------
no opt median time: 0.23064863681793213
mode = defualt: 0.1922684907913208
mode = reduce overhead: 0.19235754013061523
mode = max autotune: 0.1908893585205078
-------------Mean Time---------------
no opt mean time: 0.23183903694152833
mode = defualt: 0.19224832057952881
mode = reduce overhead: 0.19236133098602295
mode = max autotune: 0.1904613494873047


In [11]:
test(32)

-------------Median Time---------------
no opt median time: 0.8679666519165039
mode = defualt: 0.7437078952789307
mode = reduce overhead: 0.7450851202011108
mode = max autotune: 0.7439479827880859
-------------Mean Time---------------
no opt mean time: 0.9924789428710937
mode = defualt: 0.8488427639007569
mode = reduce overhead: 0.8483895063400269
mode = max autotune: 0.9141491174697876


In [12]:
test(64)

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 1368, in aot_wrapper_dedupe
    fw_metadata, _out = run_functionalized_fw_and_collect_metadata(flat_fn)(
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 569, in inner
    flat_f_outs = f(*flat_f_args)
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 2415, in functional_call
    out = Interpreter(mod).run(*args[params_len:], **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 136, in run
    self.env[node] = self.run_node(node)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 177, in run_node
    return getattr(self, n.op)(n.target, args, kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 294, in call_module
    return submod(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torc

-------------Median Time---------------
no opt median time: 1.7157058715820312
mode = defualt: 1.477898120880127
mode = reduce overhead: 1.4723057746887207
mode = max autotune: 1.4827030897140503
-------------Mean Time---------------
no opt mean time: 1.7151725053787232
mode = defualt: 1.4786361694335937
mode = reduce overhead: 1.4722907781600951
mode = max autotune: 1.4845173358917236


In [13]:
test(128)

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 1368, in aot_wrapper_dedupe
    fw_metadata, _out = run_functionalized_fw_and_collect_metadata(flat_fn)(
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 569, in inner
    flat_f_outs = f(*flat_f_args)
  File "/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py", line 2415, in functional_call
    out = Interpreter(mod).run(*args[params_len:], **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 136, in run
    self.env[node] = self.run_node(node)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 177, in run_node
    return getattr(self, n.op)(n.target, args, kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/fx/interpreter.py", line 294, in call_module
    return submod(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torc

-------------Median Time---------------
no opt median time: 3.4792494773864746
mode = defualt: 3.0186045169830322
mode = reduce overhead: 3.0502309799194336
mode = max autotune: 3.0051430463790894
-------------Mean Time---------------
no opt mean time: 3.516086387634277
mode = defualt: 3.1154479503631594
mode = reduce overhead: 3.2574779987335205
mode = max autotune: 3.2143298387527466
